# Example 1: Create a disk model from scratch and run dust continuum RT simulation.

In [1]:
import numpy as np

import astromugs.modeling as modeling
import astromugs.dust as dust

from astromugs.constants.constants import mu, autocm, amu, Ggram, kb, M_sun

### Create an instance named model

In [2]:
model = modeling.Structure()  

### Show envelope and disk default parameters:

In [3]:
model.params

Field,Type,Value,Description
acc_rate,float,1.000e-05,[Msun yr-1] Accretion rate
cav_fact,float,1.000e+01,Factor of decrease of the density in the cavity
cavpl,float,1.000e+00,Opening angle and shape of the outflow
cavz0,float,1.000e+01,[AU]
coordsystem,Literal,'spherical',Coordinate system for radiative transfer
dtogas,float,1.000e-02,Dust to gas mass ratio
dust_env_mass,float,1.000e-05,[Msun] Total dust mass of the envelope
r_centri,float,1.000e+02,[AU] Centrifugal radius (critical radius inside of which the envelope flattens)
rmax,float,5.000e+03,[AU] Outer radius of the envelope)
rmin,float,1.000e+00,[AU] inner radius of the envelope)


In [4]:
# Since we only want to create a disk, we can also show the disk parameters only:
model.params.disk

Field,Type,Value,Description
acc_rate,float,1.000e-05,[Msun yr-1] Accretion rate
alpha,float,1.000e-02,Viscosity coefficient
coordsystem,Literal,'spherical',Coordinate system for radiative transfer
d_exp,float,3.500e+00,Size distribution exponent. WARNING: don't type 4
disk_mass,float,1.500e-01,[Msun] Total mass of the disk
dtogas,float,1.000e-02,Dust to gas mass ratio
dust_mass,float,1.000e-04,[Msun] Total dust mass in the disk
h0,float,1.000e+01,[AU] Scale height at reference
isothermal,bool,False,Is vertically isothermal. Default is non-isothermal
lim_h,float,1.000e+00,Value in fraction of scale height until which accretion heating is computed


### Change the value of a parameter:

In [5]:
model.params.disk.ref_radius = 150

In [6]:
# see the value correctly replaced:
model.params.disk

Field,Type,Value,Description
acc_rate,float,1.000e-05,[Msun yr-1] Accretion rate
alpha,float,1.000e-02,Viscosity coefficient
coordsystem,Literal,'spherical',Coordinate system for radiative transfer
d_exp,float,3.500e+00,Size distribution exponent. WARNING: don't type 4
disk_mass,float,1.500e-01,[Msun] Total mass of the disk
dtogas,float,1.000e-02,Dust to gas mass ratio
dust_mass,float,1.000e-04,[Msun] Total dust mass in the disk
h0,float,1.000e+01,[AU] Scale height at reference
isothermal,bool,False,Is vertically isothermal. Default is non-isothermal
lim_h,float,1.000e+00,Value in fraction of scale height until which accretion heating is computed


### Create a dust model

In [9]:
d = dust.MRNDistrib(rsingle=0.1, amin=5.000e-03, amax=1.000e+03, nb_sizes=16, rho_m=2.5) #microns

# and add the dust model to the grid:
model.grid.add_dust(d)

### read the dust sizes

In [1]:
sizes = model.grid.dust[0].sizes()
sizes[-1]

now show the thermal parameters. Note that some parameters exist already in StructParams. 

Why twice? Because sometimes the users
wants to create radmc3d from an existing model (from chemistry or MHD), and they don't need to create a physical disk/envelope structure.

For consistency, you have to make sure the rodondant values are the same, otherwise the postprocessing can become unconsistent!